In [ ]:
# backoff ngram
import os
import random
from gt_ngram import gt_ngram
import operator
import preprocess
from math import pow, log, exp
import numpy as np

import li_gram


class bo_ngram(gt_ngram):
    def __init__(self,content,r = [1/3,1/3,1/3]):
        super(bo_ngram,self).__init__(content)
        self.r = np.array(r)
    
    def backoff_interpolation(self,prob):
    # input: lambd - estimator array
        pass

# only one iteration
    def generate_perplexity(self, n, sentences):
        for z in xrange(n):
            self.nprob_dic[1] = self.nprob_dic[1] if 1 in self.nprob_dic else self.generate_ngram(1)
            z = n+1
            self.nprob_dic[z] = self.nprob_dic[z] if z in self.nprob_dic else self.generate_ngram(z)
        tokens = preprocess.preprocess_text(sentences).split()

        # use unk_1 to repalce word not in ncounter_dic[1]
        self.ncounter_dic[1] = self.ncounter_dic[1] if 1 in self.ncounter_dic else self.ntoken_count(1)
        for i, token in enumerate(tokens):
            key = tuple([token])
            if key not in self.ncounter_dic[1]:
                tokens[i] = '<unk_1>'

        # calculate perplexity
        perp = 0
        
        _len = len(tokens)
        for i in xrange(_len):
            prob = 0 
            prob_tup = []
            for j in xrange(n):
                # ntemp: helps get all unigram to n-gram
                ntemp = j + 1
                #different
                key = tuple(tokens[i:i+ntemp])
                if n>1:
                    unk = '<unk_{}>'.format(n - 1)
                    if key not in self.nprob_dic[n]:
                        key = tuple([unk, token])
                        
                prob = self.nprob_dic[n][tuple(tokens[0:0+ntemp])] if i<ntemp else self.nprob_dic[ntemp][key]
                prob_tup.append(prob)
            prob_tup = np.array(prob_tup)
            prob = prob_tup.dot(self.r)
            perp -= log(prob)
        perp = exp(1.0 * perp / len(tokens))
        return perp

In [1]:
from ngram import ngram
import os
import preprocess
from li_ngram import li_ngram
import sys
import operator
import csv
import numpy as linspace
from bo_ngram import bo_ngram

# TODO: delete it when not used

indir_pre = os.getcwd() + "/"
outdir_pre = os.getcwd() + "/"
topics = {'atheism':0, 'autos':1, 'graphics':2, 'medicine':3, 'motorcycles':4, 'religion':5, 'space':6}

def random_sentence_ngram(n = 2, sent_pre = "I have"):
    for topic in topics:
        indir = indir_pre + "data/classification_task/{}/train_docs".format(topic)
        content = preprocess.preprocess_dir(indir)
        ngrams = ngram(content)
        print "\n\n\nTopic: {}\n".format(topic)
        for k in xrange(1, n + 1):
            print "[{}-gram]\n".format(k)

            print "Empty sentence"
            for i in xrange(3):
                print "[{}]  ".format(i + 1) + ngrams.generate_sentence(k)

            print "\nWith incomplete sentence: " + "\"{}\"".format(sent_pre)
            for i in xrange(3):
                print "[{}]  ".format(i + 1) + ngrams.generate_sentence(k, sent_pre)


def generate_perplexity_bo_ngram():
    bo_ngrams = {}
    for topic in topics:
        indir = indir_pre + "data/classification_task/{}/train_docs".format(topic)
        content = preprocess.preprocess_dir(indir)
        bo_ngrams[topic] = bo_ngram(content)

        print "\nTopic: {}".format(topic)
        print "[{}-gram]: {}".format(3, bo_ngrams[topic].generate_perplexity(3, content))


def topic_classification_bo_ngram():
    """
    calculate the accuracy for topic classification with different
    n in Good-Turing ngram, then choose the best one to classify files
    in test_for_classification directory, and write results into
    bo_result.csv in classification_task directory
    """

    # get bo_ngram for each topic and read all test data
    bo_ngrams, train_text, test_text  = {}, {}, {} #key: topic
    for topic in topics:
        train_f = indir_pre + "data/classification_task/{}/train.txt".format(topic)
        test_f = indir_pre + "data/classification_task/{}/train.txt".format(topic)
        if not os.path.isfile(train_f) or not os.path.isfile(test_f):
            split_train_test()

        train_text[topic] = open(train_f, 'r').read()
        test_text[topic] = open(test_f, 'r').read()

        bo_ngrams[topic] = bo_ngram(train_text[topic])

    # calculate the accuracy for n-gram and choose the best one
    accuracy = {} # key: the n in bo_ngram
    for i in xrange(3,4):
        _sum, correct = 0, 0
        for label_topic, text in test_text.items():
            sentences = text.split('</s>')
            for sentence in sentences:
                sentence += ' </s>'
                min_perp, min_topic = sys.maxint, label_topic

                for topic in topics:
                    perp = bo_ngrams[topic].generate_perplexity(i, sentence)
                    if perp < min_perp:
                        min_perp = perp
                        min_topic = topic

                if label_topic == min_topic:
                    correct += 1
                _sum += 1

        accuracy[i] = 1.0 * correct / _sum
        print "[{}-gram] {}".format(i, accuracy[i])
    #choose the best n
    n = 3

    # get the result for files in test_for_classification directory
    test_dir = indir_pre + "data/classification_task/test_for_classification"
    csv_f = indir_pre + "data/classification_task/bo_result.csv"

    with open(csv_f, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = ['ID', 'Prediction'])
        writer.writeheader()

        for root, dirs, filenames in os.walk(test_dir):
            for f in filenames:
                text = preprocess.preprocess_file(os.path.join(root, f))
                min_perp, min_topic = sys.maxint, ''

                for topic in topics:
                    perp = bo_ngrams[topic].generate_perplexity(n, text)
                    if perp < min_perp:
                        min_perp = perp
                        min_topic = topic

                writer.writerow({'ID': f, 'Prediction': '{}'.format(topics[min_topic])})


def split_train_test():
    """
    split train_docs into     training:test = 4:1
    store the preprocessed file train.txt and test.txt in each topic directory
    """
    for topic in topics:
        indir = indir_pre + "data/classification_task/{}/train_docs".format(topic)
        content = preprocess.preprocess_dir(indir)
        tokens = content.split()

        # find the nearest </s> after 80% content
        pointer = int(len(tokens) * 0.8)
        while tokens[pointer] != '</s>':
            pointer += 1

        train_text = ' '.join(tokens[:(pointer+1)])
        test_text = ' '.join(tokens[(pointer+2):])

        train_path = indir_pre + "data/classification_task/{}/train.txt".format(topic)
        test_path = indir_pre + "data/classification_task/{}/test.txt".format(topic)
        open(train_path, 'w').write(train_text)
        open(test_path, 'w').write(test_text)

def spell_checker_gt_nrgam():
    pass


def main():
    generate_perplexity_bo_ngram()


if __name__ == "__main__":
    main()



Topic: motorcycles
69460


AttributeError: 'dict' object has no attribute 'kasdfadsfaeys'